# Imports

In [ ]:
!nvidia-smi

In [ ]:
!pip install -Uqq datasets transformers[sentencepiece]
!pip install -Uqq accelerate
!sudo apt install -Uqq git-lfs
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl


     |████████████████████████████████| 325 kB 5.4 MB/s 
     |████████████████████████████████| 3.8 MB 40.0 MB/s 
     |████████████████████████████████| 1.1 MB 35.0 MB/s 
     |████████████████████████████████| 134 kB 31.7 MB/s 
     |████████████████████████████████| 67 kB 3.1 MB/s 
     |████████████████████████████████| 212 kB 33.1 MB/s 
     |████████████████████████████████| 127 kB 39.9 MB/s 
     |████████████████████████████████| 144 kB 35.9 MB/s 
     |████████████████████████████████| 271 kB 22.4 MB/s 
     |████████████████████████████████| 94 kB 1.7 MB/s 
     |████████████████████████████████| 596 kB 29.7 MB/s 
     |████████████████████████████████| 895 kB 31.4 MB/s 
     |████████████████████████████████| 6.5 MB 9.4 MB/s 
     |████████████████████████████████| 1.2 MB 10.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 

In [ ]:
!pip install -Uqq wandb

import wandb
wandb.login()

In [ ]:
%env WANDB_PROJECT=NLMUT

# Dataset 

## Nepali Wikipedia

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                         title                                             size  lastUpdated          downloadCount  
----------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  
prasertk/netflix-subscription-price-in-different-countries  Netflix subscription fee in different countries    3KB  2022-01-15 07:06:09           3595  
nkitgupta/jigsaw-regression-based-data                      Jigsaw Regression Based Data                       3GB  2022-02-05 20:51:56            914  
yasserh/wine-quality-dataset                                Wine Quality Dataset                              21KB  2022-01-15 19:15:11           3731  
majyhain/height-of-male-and-female-by-country-2022          Height of Male and Female by Country 2022          4KB  2022-02-02 00:40:19           1252  
sanjeetsinghnaik/top-1000-highest-grossing-movie

In [ ]:
! kaggle datasets download -d disisbig/nepali-wikipedia-articles

 49% 14.0M/28.4M [00:00<00:00, 146MB/s]
100% 28.4M/28.4M [00:00<00:00, 139MB/s]


In [ ]:
!unzip nepali-wikipedia-articles.zip

In [ ]:
from datasets import load_dataset
wiki_dataset = load_dataset("text", data_files=["train/train/*.txt", "valid/valid/*.txt"], split="train")
# wiki_dataset_with_test = load_dataset("text", data_files={"train": "train/train/*.txt", "test": "valid/valid/*.txt"})

Resolving data files:   0%|          | 0/38757 [00:00<?, ?it/s]

Using custom data configuration default-2a495049fddb51ef


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-2a495049fddb51ef/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4. Subsequent calls will reuse this data.


In [ ]:
wiki_dataset

Dataset({
    features: ['text'],
    num_rows: 157329
})

## cc100 and OSCAR

In [ ]:
# Takes 10-15 Minutes on Google Colab
from datasets import load_dataset, interleave_datasets, concatenate_datasets
from itertools import islice
deduplicated_ne = load_dataset('oscar', "unshuffled_deduplicated_ne", split='train', streaming=False)
original_ne = load_dataset('oscar', "unshuffled_original_ne", split='train', streaming=False)
cc100_ne = load_dataset("cc100", split='train', lang="ne", streaming=False)

Downloading:   0%|          | 0.00/5.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/359k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

Using custom data configuration ne-lang=ne


Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset cc100 downloaded and prepared to /root/.cache/huggingface/datasets/cc100/ne-lang=ne/0.0.0/8159941b93eb06d0288bb80be26ddfe8213c0c5e33286619c85ad8e1ee0eb91c. Subsequent calls will reuse this data.


In [ ]:
sample = cc100_ne.shuffle(seed=42).select(range(5))
print(cc100_ne)
sample[:3]

Dataset({
    features: ['id', 'text'],
    num_rows: 12732810
})


{'id': ['11611102', '4039765', '7131584'],
 'text': ['धनगढीमा कुटपिटबाट १ जनाको मृत्यु, रक्ताम्य घाइतेलाई घिसार्दै गरेको अवस्थामा १ युवक पक्राउ\n',
  'उन्नतिले एक लाख ६५ हजार कित्ता साधारण शेयर निष्कासन गरेको थियो । जसमा आठ हजार २५० कित्ता शेयर सामूहिक लगानी कोष र छ हजार ६०० कित्ता कर्मचारीलाई छुट्याएर बाँडफाँड गरिएको कम्पनीले जनाएको छ । भैरहवामा केन्द्रीय कार्यालय रहेको सो माइक्रो फाइनान्सले विभिन्न १० जिल्लामा सेवा दिइरहेको छ ।\n',
  'युक्रेनी सेना र रुस समर्थक पृथकतावादीहरुबीच विगत आठ महिनायता भएका संघर्षमा चार हजार मानिसको ज्यान गइसकेको छ।\n']}

In [ ]:
sample = original_ne.shuffle(seed=42).select(range(5))
sample[:3]

{'id': [233291, 116507, 62491],
 'text': ['मेष ( च, चू, चे, ला, ली, लू, ले, लो, अ, आ ) बच्चाहरुसँग खेल्नाले तपाईंलाई धेरै शान्तिको अनुभव हुनेछ।\tहतारमा निवेश नगर्नुहोस्। यदि निवेश गर्दा तपाईंले सबै सम्भव कोणबाट हेर्नु भएन भने घाटा निश्चित छ।\tपरिवारको तनाउले तपाईंको ध्यान नबाँटोस्। नराम्रो समयले हामीलाई धेरै ज्ञान दिन्छ। आत्मग्लानिमा संलग्न भएर समय बर्बाद नगर्नुहोस् तर जीवनबाट केही सिक्ने प्रयास गर्नुहोस्।\tआफ्नो प्रियसँग तपाईंको व्यक्तिगत भावना / रहस्य साझेदारी गर्ने यो सही समय होइन।\tतपाईंका प्रयासहरूका लागि मान्छेले काममा तपाईंलाई पहिचान गर्नेछन्।\tआज तपाईंले महत्त्वपूर्ण विषयहरूमा ध्यान केन्द्रित गर्नुपर्छ।\tतपाईंले आज आफ्नो वैवाहिक जीवनका एउटा कठिन समयको सामना गर्नुपर्ने हुन सक्छ।\t\nवृष ( ई, उ, ए, ऐ, ओ, व, वा, वो, वे, वौ, वं ) खानाको आफ्नो स्वाद नुनको बलले हुन्छ - केही दुख जरूरी हुन्छ अनि मात्र तपाईंले आनन्दको मूल्य महसुस गर्नुहुनेछ।\tविगतका लगानीबाट आयमा वृद्धि हुने देखिन्छ।\tआज तपाईंलाई लाभ हुनेछ - किनकि परिवारका सदस्यहरूले सकारात्मक ढंगमा प्रतिक्रिया गर्नेछन्।\tआज अचानक रोमान्

In [ ]:
cc100_ne

Dataset({
    features: ['id', 'text'],
    num_rows: 12732810
})

## Concatenate Datasets

In [ ]:
# Delete the 'id' column
cc100_ne = cc100_ne.map(remove_columns=['id'], batched=True)
original_ne = original_ne.map(remove_columns=['id'], batched=True)
deduplicated_ne = deduplicated_ne.map(remove_columns=['id'], batched=True)

  0%|          | 0/12733 [00:00<?, ?ba/s]

  0%|          | 0/300 [00:00<?, ?ba/s]

  0%|          | 0/220 [00:00<?, ?ba/s]

In [ ]:
combined_dataset = concatenate_datasets([cc100_ne, deduplicated_ne, original_ne, wiki_dataset])
print(list(islice(combined_dataset, 2)))
# combined_dataset_with_oversampling = interleave_datasets([deduplicated_ne, original_ne], probabilities=[0.8, 0.2], seed=42)
# print(list(islice(combined_dataset_with_oversampling, 2)))

[{'text': 'वैशाख २१ – आर्सनललाई हराउँदै एथ्लेटिको मड्रिड युरोपा लिगको फाइनलमा प्रवेश गरेको छ ।\n'}, {'text': 'घरेलु मैदानमा भएको च्याम्पियन्स लिगको दोस्रो लेगमा एथ्लेटिको मड्रिडले आर्सनललाई एक शून्यले हराउँदै समग्रमा दुई एकको अग्रताका साथ फाइनलमा प्रवेश गरेको हो ।\n'}]


In [ ]:
combined_dataset

Dataset({
    features: ['text'],
    num_rows: 13409411
})

In [ ]:
# Full Dataset
combined_dataset = combined_dataset.train_test_split(test_size=0.02, seed=42)
combined_dataset

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/cc100/ne-lang=ne/0.0.0/8159941b93eb06d0288bb80be26ddfe8213c0c5e33286619c85ad8e1ee0eb91c/cache-80a2a87b8a2f1e9c.arrow and /root/.cache/huggingface/datasets/cc100/ne-lang=ne/0.0.0/8159941b93eb06d0288bb80be26ddfe8213c0c5e33286619c85ad8e1ee0eb91c/cache-7a674f2b75d99f54.arrow


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 13141222
    })
    test: Dataset({
        features: ['text'],
        num_rows: 268189
    })
})

In [ ]:
# Dataset for OSCAR_original [100k samples] and Wiki ## For Tokenizer
dataset_train_test = combined_dataset.train_test_split(test_size=0.02, seed=42)
dataset_train_test

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 222782
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4547
    })
})

## Export Dataset

In [ ]:
from huggingface_hub import notebook_login 

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
combined_dataset.push_to_hub("Sakonii/nepalitext-language-model-dataset", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/14 [00:00<?, ?it/s]

Pushing split test to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

##  Load Straight from the HUB

In [ ]:
from huggingface_hub import notebook_login 
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from datasets import load_dataset
# combined_dataset = load_dataset('Sakonii/nepalitext-language-model-dataset', streaming=False)
# tokenized_datasets = load_dataset('Sakonii/nepali_LM_dataset_wiki_cc100_OSCAR_tokenized', streaming=False, use_auth_token=True)
lm_datasets = load_dataset('Sakonii/nepali_LM_dataset_wiki_cc100_OSCAR_lm_512', streaming=False, use_auth_token=True)

Downloading:   0%|          | 0.00/9.76k [00:00<?, ?B/s]

Using custom data configuration Sakonii--nepali_LM_dataset_wiki_cc100_OSCAR_combined-f0dbf1adbe209bdc


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/208M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/208M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/58.8M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/Sakonii--nepali_LM_dataset_wiki_cc100_OSCAR_combined-f0dbf1adbe209bdc/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

# Tokenizer

## Load Tokenizers

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, XLMRobertaTokenizer
import transformers

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base") # bert-base-uncased bert-base-multilingual-cased 
# model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

[More Details about the Model / Tokenizer](https://huggingface.co/docs/transformers/model_doc/xlm-roberta)

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [ ]:
# Retrieve the full list of tokens.
all_tokens = list(tokenizer.get_vocab().keys())

print('The vocabulary contains {:,} tokens.'.format(len(all_tokens)))

The vocabulary contains 250,002 tokens.


In [ ]:
with open("vocabulary.txt", 'w') as f:
    
    # For each token...
    for token in all_tokens:
        
        # Write it out and escape any unicode characters.            
        f.write(token + '\n')

## Generators of size 1000 
###### will be passed to train tokenizers

In [ ]:
def remove_newLineChar(line: str):
  line.replace('\t', ' ')
  return line.replace('\n', ' ')

In [ ]:
def get_training_corpus():
    return (
        # Apply remove_new_LineChar to list of string (concatenated to 1000 to form a batch)
        list(map(remove_newLineChar, combined_dataset[i : i + 1000]["text"]))
        for i in range(0, len(combined_dataset), 1000)
    )


training_corpus = get_training_corpus()

In [ ]:
# next(training_corpus)

['वैशाख २१ – आर्सनललाई हराउँदै एथ्लेटिको मड्रिड युरोपा लिगको फाइनलमा प्रवेश गरेको छ । ',
 'घरेलु मैदानमा भएको च्याम्पियन्स लिगको दोस्रो लेगमा एथ्लेटिको मड्रिडले आर्सनललाई एक शून्यले हराउँदै समग्रमा दुई एकको अग्रताका साथ फाइनलमा प्रवेश गरेको हो । ',
 'सन् २०१६ को च्याम्पियन्स लिगको फाइनलमा रियल मड्रिडसँग पराजित भएपछि एथ्लेटिको पहिलो पटक युरोपियन फुटबल प्रतियोगिताको फाइनलमा पुगेको हो । ',
 'प्रशिक्षक विङगरले युरोपियन खेलमा सफलता हात पार्न सकेनन् । प्रिमियर लिगमा राम्रो प्रदर्शन निकाल्न नसकेपछि प्रशिक्षकका रुपमा उनको ठूलो आलोचना भएको थियो । जसका कारण उनले इमिरेट्स छोड्ने निर्णयमा पुगेका थिए । ',
 'आर्सनलको प्रशिक्षकका रुपमा उनले तीन वटा प्रिमियर र सात वटा एफ ए कपको उपाधि जिते पनि युरोपियन प्रतियोगितामा भने २१ सिजनमा दुई पटक मात्र फाइनल प्रवेश गरेका छन् । ',
 'प्रतिकृया दिनुहोस ',
 'प्रधानमन्त्री ओलीले भोली गण्डकी प्रदेशसभामा सम्बोधन गर्ने ',
 'प्लास्टिक प्रयोग गरेर पोखरामा सडक पिच ',
 'नेपाल इन्जिनियर दिवस भव्यताका साथ मनाउने तयारी ',
 '‘जनतासँग प्रधानमन्त्री’ कार्यक्रम आज साँझ ९ बजेदेखि,

## Tokenizer Training

In [ ]:
tokenizer.tokenize("राष्ट्रिय सभागृहमा राप्रपा नेपाल र राप्रपा प्रजातन्त्रिक बीच २०७६ फागुन २८ मा एकीकरण कार्यक्रम चलिरहेको थियो । राजेन्द्र लिङ्देनले पार्टी एकता हुने र विभाजन हुने प्रसंग उठाएपछि तत्कालीन राप्रपा अध्यक्ष कमल थापाले गीता छोएर कसम खाँदै भनेका थिए, ‘राप्रपा कहिले पनि विभाजन हुन दिन्नँ ।")

['▁राष्ट्रिय',
 '▁सभा',
 'गृह',
 'मा',
 '▁राप्रपा',
 '▁नेपाल',
 '▁र',
 '▁राप्रपा',
 '▁प्रजातन्त्र',
 'िक',
 '▁बीच',
 '▁२०',
 '७६',
 '▁फागुन',
 '▁२८',
 '▁मा',
 '▁एक',
 'ीकरण',
 '▁कार्यक्रम',
 '▁चल',
 'िरहेको',
 '▁थियो',
 '▁।',
 '▁राजेन्द्र',
 '▁लि',
 'ङ्',
 'देन',
 'ले',
 '▁पार्टी',
 '▁एकता',
 '▁हुने',
 '▁र',
 '▁विभाजन',
 '▁हुने',
 '▁प्रसंग',
 '▁उठाए',
 'पछि',
 '▁तत्कालीन',
 '▁राप्रपा',
 '▁अध्यक्ष',
 '▁कमल',
 '▁थापाले',
 '▁गीत',
 'ा',
 '▁छ',
 'ो',
 'एर',
 '▁कस',
 'म',
 '▁खा',
 'ँदै',
 '▁भनेका',
 '▁थिए',
 ',',
 '▁‘',
 'रा',
 'प्र',
 'पा',
 '▁कहिले',
 '▁पनि',
 '▁विभाजन',
 '▁हुन',
 '▁दिन',
 '्न',
 'ँ',
 '▁।']

In [ ]:
tokenizer = tokenizer.train_new_from_iterator(training_corpus, vocab_size=24576, min_frequency=4, limit_alphabet=1000)

## Tests

In [ ]:
tokenizer.tokenize("आइतबार बेलुका ७ बजे प्रदेश प्रमुख तिलक परियारले नवनियुक्त मन्त्रीहरूलाई सपथ खुवाएका थिए। उक्त सपथको दृश्य भने नेपालको संविधानले अबलम्बन गरेको समावेशिताको सिद्धान्तको बर्खिलाप थियो। सपथ खुवाउने र खाने सबै पुरूषमात्रै थिए।")

['▁आइतबार',
 '▁बेलुका',
 '▁७',
 '▁बजे',
 '▁प्रदेश',
 '▁प्रमुख',
 '▁तिलक',
 '▁परियार',
 'ले',
 '▁',
 'नवनियुक्त',
 '▁मन्त्री',
 'हरूलाई',
 '▁सपथ',
 '▁खुवाए',
 'का',
 '▁थिए।',
 '▁उक्त',
 '▁सपथ',
 'को',
 '▁दृश्य',
 '▁भने',
 '▁नेपालको',
 '▁संविधानले',
 '▁अबलम्बन',
 '▁गरेको',
 '▁समावेशिता',
 'को',
 '▁सिद्धान्त',
 'को',
 '▁बर्',
 'खि',
 'लाप',
 '▁थियो।',
 '▁सपथ',
 '▁खुवाउने',
 '▁र',
 '▁खाने',
 '▁सबै',
 '▁पुरूष',
 'मात्रै',
 '▁थिए।']

In [ ]:
tokenizer.tokenize("राष्ट्रिय सभागृहमा राप्रपा नेपाल र राप्रपा प्रजातन्त्रिक बीच २०७६ फागुन २८ मा एकीकरण कार्यक्रम चलिरहेको थियो । राजेन्द्र लिङ्देनले पार्टी एकता हुने र विभाजन हुने प्रसंग उठाएपछि तत्कालीन राप्रपा अध्यक्ष कमल थापाले गीता छोएर कसम खाँदै भनेका थिए, राप्रपा कहिले पनि विभाजन हुन दिन्नँ ।")

['▁राष्ट्रिय',
 '▁सभागृह',
 'मा',
 '▁राप्रपा',
 '▁नेपाल',
 '▁र',
 '▁राप्रपा',
 '▁प्रजातन्त्र',
 'िक',
 '▁बीच',
 '▁',
 '२०७६',
 '▁फागुन',
 '▁२८',
 '▁मा',
 '▁एकीकरण',
 '▁कार्यक्रम',
 '▁चलिरहेको',
 '▁थियो',
 '▁।',
 '▁राजेन्द्र',
 '▁लिङ्देन',
 'ले',
 '▁पार्टी',
 '▁एकता',
 '▁हुने',
 '▁र',
 '▁विभाजन',
 '▁हुने',
 '▁प्रसंग',
 '▁उठाए',
 'पछि',
 '▁तत्कालीन',
 '▁राप्रपा',
 '▁अध्यक्ष',
 '▁कमल',
 '▁थापाले',
 '▁गीता',
 '▁छोए',
 'र',
 '▁कस',
 'म',
 '▁खाँदै',
 '▁भनेका',
 '▁थिए,',
 '▁राप्रपा',
 '▁कहिले',
 '▁पनि',
 '▁विभाजन',
 '▁हुन',
 '▁दिन',
 '्न',
 'ँ',
 '▁।']

In [ ]:
tokenizer.tokenize('पुरुष एक पुलिङ्ग हो जस्को व्यवहार र हाउभाउमा पुरुषत्व हुन्छ । \n')

['▁पुरुष',
 '▁एक',
 '▁पुल',
 'िङ्ग',
 '▁हो',
 '▁जस्',
 'को',
 '▁व्यवहार',
 '▁र',
 '▁हाउभाउ',
 'मा',
 '▁पुरुष',
 'त्व',
 '▁हुन्छ',
 '▁।']

## Export Tokenizer

In [ ]:
from huggingface_hub import notebook_login 

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
!sudo apt install git-lfs

In [ ]:
tokenizer.push_to_hub("Sakonii/distilbert-base-nepali", use_temp_dir=True, commit_message="tokenizer-wiki+OSCAR_100k")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/Sakonii/xlm-roberta-base-nepali into local empty directory.
To https://huggingface.co/Sakonii/xlm-roberta-base-nepali
   f80aa59..1f3be18  main -> main



'https://huggingface.co/Sakonii/xlm-roberta-base-nepali/commit/1f3be18e4d64904a093bfb2cdb223b2eecadc780'

In [ ]:
  # export vocab.txt
all_tokens = list(tokenizer.get_vocab().keys())

print('The vocabulary contains {:,} tokens.'.format(len(all_tokens)))

with open("vocabulary.txt", 'w') as f:
    
    # For each token...
    for token in all_tokens:
        
        # Write it out and escape any unicode characters.
        f.write(token + '\n')

The vocabulary contains 24,581 tokens.


## Straight Load from the HUB

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Sakonii/distilbert-base-nepali")

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

# Language Model

## Dataset

In [ ]:
combined_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 13141222
    })
    test: Dataset({
        features: ['text'],
        num_rows: 268189
    })
})

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(combined_dataset['train'], 3)

,text
0,नेपाली चलचित्र\n
1,\n
2,"धेरैका लागि सपनामयीजस्तो लाग्ने सैन्य पेसा र जीवनका सजिलाभन्दा अप्ठ्यारा प्रसंग कथाहरूभित्र छन्, नरेशले आफ्नो माटो/मुलुक छाडेर गएपछिको स्मृति–विस्मृति पनि कथामा उत्रिएका छन् । सैन्य जीवनका अनुभव–अनुभूति भिन्न रूपमा आए पनि यसरी आख्यानका रूपमा भने निकै कम लेखिएको थियो, यस अर्थमा नरेशले सराहनीय काम गरेका छन् ।\n"


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Sakonii/xlm-roberta-base-nepali")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
# Small Sample
combined_dataset['train'] = combined_dataset['train'].shuffle(seed=42).select(range(10000))
combined_dataset['test'] = combined_dataset['test'].shuffle(seed=42).select(range(1000))
combined_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})

In [ ]:
# Takes around 30 mins to tokenize the entire dataset on a 4-core kaggle kernel CPU
tokenized_datasets = combined_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"]) # Remove all columns, check other columns also

Token indices sequence length is longer than the specified maximum sequence length for this model (815 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (796 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2031 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1550 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2346 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

In [ ]:
# tokenized_datasets.push_to_hub("Sakonii/nepali_LM_dataset_wiki_cc100_OSCAR_tokenized", private=True)
# tokenized_datasets = load_dataset('Sakonii/nepali_LM_dataset_wiki_cc100_OSCAR_tokenized', streaming=False, use_auth_token=True)

In [ ]:
tokenized_datasets['train'][2]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'input_ids': [0,
  374,
  20207,
  5145,
  81,
  15635,
  2234,
  13948,
  2174,
  6445,
  18125,
  182,
  3160,
  182,
  14225,
  8299,
  81,
  2292,
  2678,
  2292,
  675,
  1220,
  3030,
  3480,
  11208,
  657,
  1034,
  998,
  4462,
  184,
  68,
  702,
  3097,
  4903,
  44,
  1432,
  10125,
  7695,
  8486,
  1513,
  11208,
  16514,
  3677,
  4256,
  399,
  10,
  17,
  11277,
  2]}

In [ ]:
block_size = tokenizer.model_max_length
block_size

512

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


In [ ]:
# Takes around 23 mins on the entire dataset on a 4-core kaggle kernel CPU
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
# lm_datasets.push_to_hub("Sakonii/nepali_LM_dataset_wiki_cc100_OSCAR_lm_512", private=True)
lm_datasets = load_dataset('Sakonii/nepali_LM_dataset_wiki_cc100_OSCAR_lm_512', streaming=False, use_auth_token=True)

In [ ]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 824
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 91
    })
})

In [ ]:
tokenizer.decode(lm_datasets['train'][2]["input_ids"])

'बनेको छ । पढाउन गए पनि हुने र नगए पनि हुने प्रवृत्तिलाई चिन्नुपर्छ । अर्काेतिर गुरुको सम्मानमा पनि ह्रास आएको छ । उस्तै परे शिक्षक र विद्यार्थी सँगै चुरोट र रक्सी खाइरहेका भेट्टाउन सकिन्छ ।” नवनियुक्त उपप्रधानमन्त्री एवम् शिक्षामन्त्री गोपालमान श्रेष्ठको भनाइ हो यो ।</s><s> चौतारी पुस्तक शृंखला - ८८</s><s> प्रस्तावित व्यवस्थामा विभिन्न टिकाटिप्पणी भइरहेका छन् । केहीले हटाउनुपर्ने सुझावसमेत दिएका छन् । सबैका सुझाव मनन गरेर हामीले तत्काललाई उक्त व्यवस्था झिकेर विधेयक पारित प्रक्रिया अघि बढाउने समझदारी गरेका छौँ । ”</s><s> त्यो त थाहा भएन</s><s> युवा स्वरोजगार कार्यक्रमको १२ करोड कसले खायो?</s><s> यो २ हप्ता भित्रमा गएको यो तेस्रो ठुलो भूकम्प हो,इन्डोनेसियन राष्टिय आपतकालीन ब्यबस्थापन सस्थाका प्रवक्ताका अनुसार अहिले सम्म भूकम्पबाट झन्डै ३५० जनाको मृत्यु भएको,१५०० भन्दा बढी घाइते तथा १६५,००० जना जति भूकम्पबाट प्रभावित भएको जानकारी दिइएको छ ।- एजेन्सी</s><s> नेपाली संगीत आकाशको वृहत कार्यक्रम इमेज अवार्डको स्थापना वि.स. २०५५ सालमा इमेज एफ एम ९७.९ को उदघाटन संगै भएको हो । 1st annual image a

## Training

In [ ]:
from transformers import AutoConfig, AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("Sakonii/distilbert-base-nepali", use_auth_token=True)# xlm-roberta-base

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="lm-train/",
    evaluation_strategy = "epoch",
    save_strategy="epoch",
    logging_steps=5,
    learning_rate=5e-5,
    save_steps=12000,
    weight_decay=0.01,
    load_best_model_at_end=True,
    fp16=True, 
    num_train_epochs=1,
    lr_scheduler_type='linear', 
    per_device_train_batch_size=28,
    # per_device_eval_batch_size=1,
    warmup_ratio=0.0, 
    warmup_steps=0,
    seed=42,
    push_to_hub=True,
    report_to="wandb",
    run_name="distilbert-base-ne-scratch",
    hub_model_id="Sakonii/distilbert-base-nepali",
)


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.20
    )

In [ ]:
#Spliting Dataset into 2 folds:
# 1st fold:
# lm_datasets['train'] = lm_datasets['train'].select(range(lm_datasets.num_rows['train']//2))
# lm_datasets['test'] = lm_datasets['test'].select(range(lm_datasets.num_rows['test']//2))

lm_datasets['train'] = lm_datasets['train'].shuffle(seed=42).select(range(132734, 1132734))

# Tiny Sample
# lm_datasets['train'] = lm_datasets["train"].select(range(600))
# lm_datasets['test'] = lm_datasets["test"].select(range(200))

In [ ]:
!sudo apt install git-lfs

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

Using amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 824
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 824


Epoch,Training Loss,Validation Loss


Saving model checkpoint to lm-train/checkpoint-500
Configuration saved in lm-train/checkpoint-500/config.json
Model weights saved in lm-train/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 91
  Batch size = 8


RuntimeError: ignored

In [ ]:
# Calculate perplexity and evaluation loss
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.push_to_hub()